In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df = pd.read_csv(r'C:\Users\sisir.sahu\Downloads\Machine Learning\Gradient Boosting\drug200.csv')

In [3]:
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,49,M,HIGH,NORMAL,6.269,drugA
1,47,F,NORMAL,NORMAL,6.683,drugX
2,72,M,LOW,HIGH,6.769,drugC
3,55,M,NORMAL,NORMAL,7.261,drugX
4,46,M,NORMAL,NORMAL,7.285,drugX


In [4]:
df.isnull().sum()

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


In [6]:
df1 = df.copy()

In [7]:
df['Drug'].value_counts()

DrugY    91
drugX    54
drugA    23
drugC    16
drugB    16
Name: Drug, dtype: int64

In [8]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1].values

y = y.reshape(-1, 1)

In [9]:
cat_obj = [feature for feature in x.columns if x[feature].dtypes == 'O']

In [10]:
#convert categorical to numeric objects
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder() 

for feature in cat_obj:
    # data[feature] = data[feature].values.reshape(-1,1)
    x[feature] = oe.fit_transform(x[feature].values.reshape(-1,1))

In [11]:
#split x and y into training and test dataset
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42) 

In [12]:
#scale the target variable
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test  = le.transform(y_test)

C:\Users\sisir.sahu\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [13]:
#Create Gradient Boost classifier object
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(x_train, y_train)

y_pred = gbc.predict(x_test)

In [14]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

[[18  0  0  0  0]
 [ 0  5  0  0  0]
 [ 0  1  2  0  0]
 [ 0  0  0  3  0]
 [ 0  0  0  0 11]]


In [15]:
#Accuracy Score or Recall
from sklearn.metrics import accuracy_score
ac = accuracy_score(y_test, y_pred)

print(ac)

0.975


In [16]:
#Classification report
from sklearn.metrics import classification_report
cr = classification_report(y_test, y_pred)

print(cr)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       0.83      1.00      0.91         5
           2       1.00      0.67      0.80         3
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00        11

    accuracy                           0.97        40
   macro avg       0.97      0.93      0.94        40
weighted avg       0.98      0.97      0.97        40



In [17]:
#check for bias and variance
bias = gbc.score(x_train, y_train)
variance = gbc.score(x_test, y_test)  

print(bias, variance)

1.0 0.975


In [19]:
#GRID SEARCH CV
from sklearn.model_selection import cross_val_score

parameters = {'n_estimators': np.arange(100, 500, 100),
              'learning_rate': [0.1, 0.05, 0.5]}
model = GradientBoostingClassifier()

In [20]:
from sklearn.model_selection import GridSearchCV
gsv = GridSearchCV(estimator = gbc, 
                   param_grid = parameters,
                   scoring = 'accuracy',
                   cv = 10)
cross_val_score(model, x_train, y_train, cv = 10)

array([1.    , 1.    , 1.    , 1.    , 0.9375, 1.    , 0.9375, 1.    ,
       1.    , 0.9375])